In [49]:
import requests
import requests.auth
import csv
import time

In [44]:
user_agent = "Test/0.1 by maxoboe"
def get_access_code(user_agent):
    with open('account_info.txt', 'r') as file:
        lines = file.read().splitlines()
        username = lines[0]
        password = lines[1]
        clientID = lines[2]
        secretID = lines[3]
    client_auth = requests.auth.HTTPBasicAuth(clientID, secretID)
    post_data = {"grant_type": "password", "username": username, "password": password}
    headers = {"User-Agent": user_agent}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    return response.json()['access_token']
def try_request(endpoint, access_token, headers, second_try=False):
    response = requests.get(endpoint, headers=headers)
    if 'error' in response.json():
        if second_try:
            return access_token, response
        if response.json()['error'] == 401:
            access_token = get_access_token(user_agent)
            return try_request(endpoint, access_token, user_agent,second_try=True)
    return access_token, response
def format_row(child):
    data = child['data']
    title = data['title']
    text = data['selftext']
    url = data['url']
    return {'title':title, 'text': text, 'url':url}

In [52]:
access_token = get_access_code(user_agent)
headers = {"Authorization": "bearer " + access_token, "User-Agent": user_agent}
query = "climate+change"
endpoint = "https://oauth.reddit.com/r/askscience/search?q=" + query + "&restrict_sr=on&sort=relevance&t=all&limit=100"
after = "none"
with open('raw_queries.csv', 'w+', encoding='utf-8') as csvfile:
    fieldnames = ['title', 'text', 'url']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    while after is not None:
        access_token, response = try_request(endpoint + "&after=" + after, 
                                     access_token, headers)
        after = response.json()['data']['after']
        for child in response.json()['data']['children']:
            writer.writerow(format_row(child))
        time.sleep(1.1) #Rate limited to 60 requests per minute
# Do this, but with after = after from first, count = count + 100